In [1]:
from cnp.data import PredatorPreyGenerator
import argparse
import torch

import matplotlib.pyplot as plt

In [17]:
device = torch.device('cpu')

gen =  PredatorPreyGenerator(iterations_per_epoch=1024,
                             batch_size=16,
                             x_context_ranges=[0, 100],
                             max_num_context=50,
                             min_num_target=100,
                             max_num_target=100,
                             device=device,
                             x_target_ranges=None)

In [11]:
# %time train_data = [[batch for batch in gen] for epoch in range(1)]

CPU times: user 36.9 s, sys: 0 ns, total: 36.9 s
Wall time: 36.9 s


In [ ]:
%time epoch = gen.pregen_epoch()

In [ ]:
len(epoch)

In [ ]:
task = train_data[0][0]

for idx in range(8):

    plt.scatter(task['x'][idx, :], task['y'][idx, 0, :])
    plt.scatter(task['x'][idx, :], task['y'][idx, 1, :])
    plt.show()